In [2]:
import tiff
import numpy as np
import pandas as pd
from PIL import Image
import scanpy as sc


## Align adata with region masks generated in qupath

In [ ]:

samples = {
    "B1S3": {
        "adata_path": "/staging/leuven/stg_00079/projects/analysis/LecMicro/NovaST/00.DeepSeq/MPC__527489__AB195-default_adata.h5ad",
        "image_paths": [
            "../../100.Segmentation/B1S3/B1S3_ROI_polygon_mask_raw.tif",
            "../../100.Segmentation/B1S3/B1S3_str_polygon_mask_raw.tif",
            "../../100.Segmentation/B1S3/B1S3_th_polygon_mask_raw.tif",
            "../../100.Segmentation/B1S3/B1S3_crx_polygon_mask_raw.tif",
            "../../100.Segmentation/B1S3/B1S3_hip_polygon_mask_raw.tif"
        ],
        "rotate_angle": -90,
        "mirror": False
    },
    "B1S4": {
        "adata_path": "/staging/leuven/stg_00079/projects/LecMicro/NovaST/00.DeepSeq/MPC__b261e5__AB193-default_adata.h5ad",
        "image_paths": [
            "../../100.Segmentation/B1S4/B1S4_ROI_polygon_mask_raw.tif",
            "../../100.Segmentation/B1S4/B1S4_str_polygon_mask_raw.tif",
            "../../100.Segmentation/B1S4/B1S4_th_polygon_mask_raw.tif",
            "../../100.Segmentation/B1S4/B1S4_crx_polygon_mask_raw.tif",
            "../../100.Segmentation/B1S4/B1S4_hip_polygon_mask_raw.tif"
        ],
        "rotate_angle": 90,
        "mirror": False
    },
    "B2S3": {
        "adata_path": "/staging/leuven/stg_00079/projects/analysis/LecMicro/NovaST/00.DeepSeq/MPC__6b38af__AB202-default_adata.h5ad",
        "image_paths": [
            "../../100.Segmentation/B2S3/B2S3_ROI_polygon_mask_raw.tif",
            "../../100.Segmentation/B2S3/B2S3_str_polygon_mask_raw.tif",
            "../../100.Segmentation/B2S3/B2S3_th_polygon_mask_raw.tif",
            "../../100.Segmentation/B2S3/B2S3_crx_polygon_mask_raw.tif",
            "../../100.Segmentation/B2S3/B2S3_hip_polygon_mask_raw.tif"
        ],
        "rotate_angle": -90,
        "mirror": False
    },
    "B2S4": {
        "adata_path": "/staging/leuven/stg_00079/projects/analysis/LecMicro/NovaST/00.DeepSeq/MPC__9100be__AB204-default_adata.h5ad",
        "image_paths": [
            "../../100.Segmentation/B2S4/B2S4_ROI_polygon_mask_raw.tif",
            "../../100.Segmentation/B2S4/B2S4_str_polygon_mask_raw.tif",
            "../../100.Segmentation/B2S4/B2S4_th_polygon_mask_raw.tif",
            "../../100.Segmentation/B2S4/B2S4_crx_polygon_mask_raw.tif",
            "../../100.Segmentation/B2S4/B2S4_hip_polygon_mask_raw.tif"
        ],
        "rotate_angle": 90,
        "mirror": True  # Add the mirror flag here for B2S4
    }
}

# Function to apply mask to AnnData object
def apply_mask_to_adata(mask_image, adata, roi_column_name):
    mask_array = np.array(mask_image)
    threshold = 128
    mask_binary = mask_array > threshold
    x_coords = adata.obs['x'].astype(int).values
    y_coords = adata.obs['y'].astype(int).values

    height, width = mask_binary.shape
    valid_mask = (x_coords >= 0) & (x_coords < width) & (y_coords >= 0) & (y_coords < height)

    adata.obs[roi_column_name] = False
    adata.obs.loc[valid_mask, roi_column_name] = mask_binary[y_coords[valid_mask], x_coords[valid_mask]]

# Loop through samples
for sample, sample_info in samples.items():

    adata = sc.read_h5ad(sample_info["adata_path"])
    
    # Assign x / y coordinates
    adata.obs["x"] = adata.obsm["spatial_um"][:, 0]
    adata.obs["y"] = adata.obsm["spatial_um"][:, 1]

    # process each ROI image for the current sample
    rotated_images = []
    for image_path in sample_info["image_paths"]:
        image = tiff.imread(image_path)
        image_pil = Image.fromarray(image)
        
        # rotate  image
        image_rotated = image_pil.rotate(sample_info["rotate_angle"], expand=True)
        
        # mirror the image if required (b2s4 needs mirroring)
        if sample_info["mirror"]:
            image_rotated = image_rotated.transpose(Image.FLIP_LEFT_RIGHT)
        
        # Convert back to nparray
        image_rotated_np = np.array(image_rotated)
        rotated_images.append(image_rotated_np)

    # Apply masks to the objects
    roi_columns = ['sample_ROI', 'str_ROI', 'th_ROI', 'crx_ROI', 'hip_ROI']
    for mask_image, roi_column in zip(rotated_images, roi_columns):
        apply_mask_to_adata(mask_image, adata, roi_column)

    for roi_column in roi_columns:
        adata.obs[roi_column] = adata.obs[roi_column].astype('category')

    # Create region column
    adata.obs['region'] = np.select(
        [adata.obs['str_ROI'], adata.obs['th_ROI'], adata.obs['crx_ROI'], adata.obs['hip_ROI']],
        ['str', 'th', 'crx', 'hip'],
        default='unknown'
    )

    adata.obs.loc[(adata.obs['sample_ROI'] == True) & (adata.obs['region'] == 'unknown'), 'region'] = 'sample'

    adata.write_h5ad(f'../{sample}_40um_regionAssignment.h5ad')


# merge all objects

In [ ]:
B1S3 = sc.read_h5ad('../B1S3_40um_regionAssignment.h5ad')
B1S4 = sc.read_h5ad('../B1S4_40um_regionAssignment.h5ad')
B2S3 = sc.read_h5ad('../B2S3_40um_regionAssignment.h5ad')
B2S4 = sc.read_h5ad('../B2S4_40um_regionAssignment.h5ad')


In [ ]:

objects = {
    'B1S3': B1S3,
    'B1S4': B1S4,
    'B2S3': B2S3,
    'B2S4': B2S4
}

# Iterate over dictionary, set  'sample' column
for prefix, obj in objects.items():
    obj.obs['sample'] = prefix

In [ ]:
# modify indices to make sure unique

for prefix, obj in objects.items():
    obj.obs.index = [prefix + str(i) for i in obj.obs.index]

In [ ]:
combined_adata = sc.concat([B1S3, B1S4, B2S3, B2S4])

In [ ]:
# Add human/mouse columns in var
combined_adata.var["human"] = combined_adata.var_names.str.startswith("GRCh38_")
combined_adata.var["mouse"] = combined_adata.var_names.str.startswith("mm10__")
    
sc.pp.calculate_qc_metrics(combined_adata, qc_vars=["human", "mouse"], inplace=True)

In [ ]:
combined_adata.write_h5ad('./40um_allRegions_combined.h5ad')

In [ ]:
combined_crx = combined_adata[combined_adata.obs['region'] == 'crx_ROI']
combined_crx.write_h5ad('./40um_allCrx_combined.h5ad')

In [ ]:
B1S3_crx  = B1S3[B1S3.obs['region'] == 'crx_ROI']
B1S4_crx= B1S4[B1S4.obs['region'] == 'crx_ROI']
B2S3_crx= B2S3[B2S3.obs['region'] == 'crx_ROI']
B2S4_crx= B2S4[B2S4.obs['region'] == 'crx_ROI']

B1S3_crx.write_h5ad('./40um_B1S3_crx.h5ad')
B1S4_crx.write_h5ad('./40um_B1S4_crx.h5ad')
B2S3_crx.write_h5ad('./40um_B2S3_crx.h5ad')
B2S4_crx.write_h5ad('./40um_B2S4_crx.h5ad')